In [11]:
# pip install pandas openpyxl yfinance

In [1]:
import pandas as pd
import yfinance as yf

# Read the Excel file with tickers
input_file = 'stocks.xlsx'  # File path to your Excel file
output_file = 'stock_prices_output.xlsx'  # Output file where we'll save the data

# Read the 'Ticker' column from the Excel file
df = pd.read_excel(input_file)

# Ensure there's a 'Ticker' column in the Excel
if 'Ticker' not in df.columns:
    raise ValueError("The Excel file must contain a 'Ticker' column.")

# Define the date range for fetching data
start_date = '2023-01-01'
end_date = '2024-09-30'

# Initialize a dictionary to store the data
all_data = {}

# Loop through each ticker and fetch the data
for index, row in df.iterrows():
    ticker = row['Ticker']
    print(f"Fetching data for {ticker}")
    
    # Fetch historical daily prices from Yahoo Finance
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if not stock_data.empty:
            all_data[ticker] = stock_data['Close']  # Fetch only the closing prices
        else:
            print(f"No data found for {ticker}")
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Convert the dictionary to a pandas DataFrame
# Each ticker's data will be in its own column
prices_df = pd.DataFrame(all_data)

# Save the result to a new Excel file
prices_df.to_excel(output_file, index=True)  # Include the date index

print(f"Data fetching complete. The output is saved to {output_file}.")


Fetching data for LLY
Fetching data for NFLX
Fetching data for INTU
Fetching data for META
Fetching data for MSFT
Fetching data for SMCI
Fetching data for ANET
Fetching data for PANW
Fetching data for TSLA
Fetching data for AAPL
Fetching data for AMZN
Fetching data for COIN
Fetching data for AMD
Fetching data for GOOGL
Fetching data for ARM
Fetching data for NVDA
Fetching data for SNOW
Fetching data for DIS
Fetching data for PYPL
Fetching data for BYDDY
Fetching data for SOXL
Fetching data for MARA
Fetching data for BASE
Fetching data for CLSK
Fetching data for BABA
Fetching data for JD
Fetching data for BIDU
Fetching data for PDD
Fetching data for NIO
Fetching data for XIACF
Fetching data for TCEHY
Fetching data for LI
Fetching data for XPEV
Data fetching complete. The output is saved to stock_prices_output.xlsx.


In [ ]:
import pandas as pd
import yfinance as yf
import time

# Function to clean and standardize ticker symbols for Yahoo Finance
def clean_ticker(ticker):
    parts = ticker.split()  # Split the ticker string into parts
    base_ticker = parts[0]  # The first part is the ticker symbol
    exchange = parts[1]     # The second part is the exchange identifier

    # Mapping exchange identifiers to Yahoo Finance suffixes
    exchange_suffixes = {
        'HK': '.HK',  # Hong Kong
        'TI': '.MI',  # Italy (Milan)
        'UR': '.IS',  # Turkey (Istanbul)
        'MK': '.KL',  # Malaysia (Kuala Lumpur)
        'TT': '.TW',  # Taiwan
        'JP': '.T',   # Japan
        'CN': '.TO',  # Canada
        'LN': '.L',   # United Kingdom (London)
        'GR': '.DE',  # Germany
        'FP': '.PA',  # France (Paris)
        'AU': '.AX',  # Australia
        'KS': '.KS',  # South Korea
        'IN': '.NS',  # India (NSE)
        'SP': '.SI',  # Singapore
        'SW': '.SW',  # Switzerland
        'SS': '.ST',  # Sweden
        'BZ': '.SA',  # Brazil
        'MM': '.MX',  # Mexico
        'CH': '.SS',  # China (Shanghai)
        'SZ': '.SZ',  # China (Shenzhen)
        'NA': '.AS',  # Netherlands (Amsterdam)
        'BB': '.BR',  # Belgium (Brussels)
        'SM': '.MC'   # Spain (Madrid)
    }

    # For Indian tickers, determine if NSE or BSE
    if exchange == 'IN':
        # Here I'm assuming you're using NSE tickers primarily; change if necessary
        return f"{base_ticker}.NS"

    # If the exchange is in the mapping, append the correct suffix
    if exchange in exchange_suffixes:
        return f"{base_ticker}{exchange_suffixes[exchange]}"
    
    # For US tickers (like UW, UN), remove the suffix and return the base ticker
    return base_ticker

# Function to fetch stock data from Yahoo Finance
def fetch_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'Country': info.get('country', 'N/A'),
            'Market Cap': info.get('marketCap', 'N/A'),
            'Industry': info.get('industry', 'N/A'),
            'Sector': info.get('sector', 'N/A')
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return {
            'Country': 'N/A',
            'Market Cap': 'N/A',
            'Industry': 'N/A',
            'Sector': 'N/A'
        }

# Read the Excel file
input_file = 'WLS as of Sep 20 20241.xlsx'  # Change this to your file path
output_file = 'WSL pivot 1.xlsx'  # File to save the results
batch_size = 100  # Set batch size for processing (adjust as needed)
save_interval = 1  # Save progress after every batch

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(input_file)

# Check if the file contains a "Ticker" column
if 'Ticker' not in df.columns:
    raise ValueError("The Excel file must contain a 'Ticker' column.")

# Initialize new columns for the data we want to fill
if 'Country' not in df.columns:
    df['Country'] = ''
if 'Market Cap' not in df.columns:
    df['Market Cap'] = ''
if 'Industry' not in df.columns:
    df['Industry'] = ''
if 'Sector' not in df.columns:
    df['Sector'] = ''

# Loop through the tickers and fetch the data in batches
for batch_start in range(0, len(df), batch_size):
    batch_end = min(batch_start + batch_size, len(df))
    print(f"Processing batch: {batch_start} to {batch_end}")

    for index, row in df.iloc[batch_start:batch_end].iterrows():
        ticker = row['Ticker']
        if pd.isna(ticker):
            continue
        
        # Clean the ticker symbol (remove suffixes)
        cleaned_ticker = clean_ticker(ticker)

        # Fetch stock data
        stock_data = fetch_stock_data(cleaned_ticker)

        # Assign the fetched data to the dataframe
        df.at[index, 'Country'] = stock_data['Country']
        df.at[index, 'Market Cap'] = stock_data['Market Cap']
        df.at[index, 'Industry'] = stock_data['Industry']
        df.at[index, 'Sector'] = stock_data['Sector']
    
    # Save progress after processing each batch
    df.to_excel(output_file, index=False)
    print(f"Batch {batch_start} to {batch_end} saved.")

    # Optional: Add a delay between batches to avoid overloading the API (adjust as needed)
    time.sleep(1)

print("Data processing completed.")
